In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from zipfile import ZipFile 
  
with ZipFile('/kaggle/input/quora-insincere-questions-classification/embeddings.zip', 'r') as embd_zip: 
    print(embd_zip.namelist())


1. * **DEBUT DU TRAVAIL SUR LES DONNES**

In [ ]:
def DEBUG_DICTIONARY(dct, limit=10):
    for i, key in enumerate(dct.keys()):
        if i > limit: break
        print(key, dct[key])

In [ ]:
from sklearn.model_selection import train_test_split

# conguration des donnees d entrainement et de test
train_data, val_data = train_test_split(pd.read_csv('/kaggle/input/quora-insincere-questions-classification/train.csv'), test_size=0.2, random_state=42)
sentences, targets = train_data['question_text'], train_data['target']
val_sentences, val_targets = val_data['question_text'], val_data['target']

In [ ]:
train_data.head()

In [ ]:
targets.value_counts(), val_targets.value_counts()

In [ ]:
# Pour chaque mot - compter combien de il occure dans les phrases
def configure_sentences(sentences, lower = True):
    words = {}
    for sentence in sentences:
        for word in sentence.split():
            if lower: word = word.lower()
            words[word] = words.get(word, 0) + 1
    return words

words = configure_sentences(sentences)
#frequence des mot dans les phrase 
DEBUG_DICTIONARY(words)

In [ ]:
# coconnaitre quelle type de mot est le plus utilisee
DEBUG_DICTIONARY({word: cnt for word, cnt in sorted(words.items(), key=lambda item: item[1], reverse=True)})

In [ ]:
# indexage des mot
def configure_words(words):
    vocabulary = {}
    for i, word in enumerate(words.keys()):
        vocabulary[word] = i # vocabulary[i] = word
    return vocabulary

vocabulary = configure_words(words)
DEBUG_DICTIONARY(vocabulary)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# retourne le minum le maximum le moyen la longueur et trace aussi l histograme de distrubission de longueur  
def configure_sentence_statistic(sentences):
    def sentence_len(s):
        return len(s.split())
    
    sentences.apply(sentence_len).plot(title='Sentence Length Distribution',y='Length Frequency',kind='hist', colormap='autumn', logy=True);
    return np.min(sentences.apply(sentence_len)), np.round(np.mean(sentences.apply(sentence_len))), np.max(sentences.apply(sentence_len))

min, avrg, max = configure_sentence_statistic(sentences)

print('minimum sentence length {} - average sentence length {} - maximum sentence length {}'.format(min, avrg, max))

In [ ]:
HIDDEN_SIZE = 30

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

# Affiche comment target est distrubie 
def configure_target_statistic(targets):
    trg_cnt = targets.value_counts()
    labels, sizes = (np.array(trg_cnt.index)), (np.array(100*(trg_cnt/trg_cnt.sum())))
    py.iplot(go.Figure(data=[go.Pie(labels=labels, values=sizes)], layout=go.Layout(title='Distrubition de target',font=dict(size=15),width=500, height=500)))
    return trg_cnt

configure_target_statistic(targets)

In [ ]:
# filttrer les donnees suivant les parametre fourni
def filter_and_display_data(sentences, targets, target=0, min_len=5, max_len=30, limit=3):
    result = []
    for i, sentence in enumerate(sentences):
        sent_len = len(sentence.split(' '))
        if min_len <= sent_len and sent_len <= max_len:
            if targets[i] == target:
                result.append(sentence)
                if len(result) >= limit: break
    
    if(len(result) ==- 0):
        print('no such sequencies found.')
        return
    
    print('{} {} sentences with length between {}-{}:\n'.format(limit, 'GOOD' if target == 0 else 'BAD', min_len, max_len))
    for i, s in enumerate(result):
        print(str(i+1)+")",s)

Affichage des expemple de nos donnees

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=0)

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=1)

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=0, min_len=120, max_len=140)

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=1, min_len=120, max_len=140)

Pour une longueur entre 120 et 140 il parrait que nos donnees sont difficile a determiner target meme pour un humain et pour target
=1 on n a pas d exemples  donc il est pas bon pour nos donnnees d avoir toute les types des exemples basic mais ca ne fait rien 

In [ ]:
# lit et retourne un dictionnaire cle: mot;ads and returns dictionary - key: word; valeur le vecteur attachee  (vec. length=300)
def confnigure_embeddings(embd_path):
    word2vecs = {}
    with ZipFile('/kaggle/input/quora-insincere-questions-classification/embeddings.zip') as embd_zip:
        for embd in embd_zip.open(embd_path, 'r'):
            word2vec = embd.decode().split(' ')
            word2vecs[word2vec[0]] = np.asarray(word2vec[1:], dtype='float32')
    return word2vecs
            
word2vecs = confnigure_embeddings('glove.840B.300d/glove.840B.300d.txt')
DEBUG_DICTIONARY(word2vecs, limit=1)

In [ ]:
#Dans chaque phrase remplace les mots par les  in each sentence replaces words with its own embedding vectors 
def configure_word2vecs(sentences, word2vecs):
    def configure_sentence(sentence, len=HIDDEN_SIZE):
        return ([word2vecs.get(word.lower(), np.zeros(300)) for word in sentence.split()] + [np.zeros(300)]*len)[:len] 
    
    return [configure_sentence(sentence) for sentence in sentences]

# embedding_sentences = configure_word2vecs(sentences, word2vecs)
# print(embedding_sentences[:10])

Debut de travail sur le modele 

In [ ]:
import torch
import torch.nn as nn

import seaborn as sns
import numpy as np
import pandas as pd

In [ ]:
BATCH_SIZE = 256
BATCHES = (len(sentences)+BATCH_SIZE-1)//BATCH_SIZE

EPOCHS = 2 # gpu :(
EMBD_SIZE = 300

In [ ]:
gpu = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpu, torch.cuda.is_available()

In [ ]:
#la longue courte memoire est la plus en adequation pour ce cas comme notre donnes d entrainemrnt est plus 1M et nos vecteurs attachee sont
#de longueur 300 si nous convertons notre toute notre data dans tensor une fois on a besin de 16 gb de de ram et beauccoup plus de ressources pour entrainer cette data

# Aussi nous utilisons une couche lineaire car on a pas des dependences difficile -comme on a dans tous les qui contient bad une target de 1 le plus souvent 


class LSTM(nn.Module):
    def __init__(self, input_dim=1, emb_dim=EMBD_SIZE, hid_dim=HIDDEN_SIZE, n_layers=1, output_dim=1, dropout=0.3):
        super().__init__()
        self.hid_dim, self.n_layers = hid_dim, n_layers
        
        # nn's
        self.lstm = nn.LSTM(emb_dim, hid_dim, n_layers, batch_first=True)
        self.linear = nn.Linear(hid_dim, output_dim)
        
       
        
        
    def forward(self, src):
        outputs, (hidden, cell) = self.lstm(src)
        return self.linear(hidden.reshape(-1, self.hid_dim))


In [ ]:
# craete model - with lstm and linear layers
model = LSTM().to(gpu)

# init loss function
loss_function = nn.BCEWithLogitsLoss().to(gpu) #nn.MSELoss()

# init optimizer with learning rate 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model

In [ ]:
#evalue et retourne l accuracy pour Y predit par le modele
def acc_function(y_pred, y_test):
    y_pred = torch.round(torch.sigmoid(y_pred).to(gpu)).to(gpu)
    correct = (y_pred == y_test).sum().float()
    return torch.round(100*(correct/y_pred.shape[0]))

# genere et retoune idx-ieme batch conmme torch tensor suivant les donneees fournies
def get_batch(sentences, targets, idx):
    src = configure_word2vecs(sentences[BATCH_SIZE*idx:BATCH_SIZE*(idx+1)], word2vecs)
    trg = np.asarray(targets[BATCH_SIZE*idx:BATCH_SIZE*(idx+1)], dtype='bool')
    return torch.FloatTensor(src).to(gpu), torch.FloatTensor(trg).to(gpu)

#evalue et retourne f1 pour y predis par le model
def f1_score(y_pred, y_test):
    tp = (y_test * y_pred).sum().to(torch.float32)
    tn = ((1 - y_test) * (1 - y_pred)).sum().to(torch.float32)
    fp = ((1 - y_test) * y_pred).sum().to(torch.float32)
    fn = (y_test * (1 - y_pred)).sum().to(torch.float32)
    
    epsilon = 1e-7 #pour feviter les crash
    precision, recall = tp / (tp + fp + epsilon), tp / (tp + fn + epsilon)
    
    return 2*(precision*recall)/(precision + recall + epsilon)
    

DEBUT D ENTRAINEMET DU MODELE

In [ ]:
# pret pour l entrainement
model.train()

VALIDATION_BATCHES = 10
# validation du data pour la precision pendant l entrainement -mais prenant seulement VALIDATIONBATCHES pendant que toute la data est tres grande.
val_sents = configure_word2vecs(val_sentences[:VALIDATION_BATCHES*BATCH_SIZE], word2vecs)
val_targs = np.asarray(val_targets[:VALIDATION_BATCHES*BATCH_SIZE], dtype='bool')

val_batch = torch.FloatTensor(val_sents).to(gpu)
val_target = torch.FloatTensor(val_targs).to(gpu)
print(type(val_batch), val_batch.shape, type(val_targets), val_targets.shape)

In [ ]:
BATCHES, BATCH_SIZE, get_batch(sentences, targets, 0)[0].shape, get_batch(sentences, targets, 0)[1].shape

In [ ]:
# entrainement 
for e in range(EPOCHS):
    # enregistre epoch loss et accuracy
    epoch_loss, epoch_acc = 0, 0
    for b in range(BATCHES):
        # prend le batch courant de la data
        X_batch, y_batch = get_batch(sentences, targets, b)
        
        # met le gradientsset a zero,avant de commencer la backpropagation - evitant la msnwue de direction pour le minumum .
        optimizer.zero_grad()

        #predit targets pour le bstch courant et apprend en la comparant avec la fonction loss.
        y_pred = model(X_batch)
        loss = loss_function(y_pred, y_batch.unsqueeze(1))
        
        #predit target pour la data de validation et evalue la precision
        val_pred = model(val_batch)
        acc = acc_function(val_pred, val_target.unsqueeze(1))

        #grandients sont enregistree par tensors-lors de l appel de backward sur la fonction loss.
        loss.backward()
        
        #met a jour les parametres du modele
        optimizer.step()
        
        #ajoute bash et accuracy pour evaluer epoch loss/acc.
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
        if b == 0 or (b+1) % 100 == 0:
            print(f'Epoch {(e+1)+0:03} | Batch {(b+1)+0:04}: | Loss: {epoch_loss/(b+1):.5f} | Acc: {epoch_acc/(b+1):.3f} | F1: {f1_score(val_pred, val_target.unsqueeze(1)):.3f}')
            
    print(f'Epoch {(e+1)+0:03}: | Epoch Loss: {epoch_loss/BATCHES:.5f} | Epoch Acc: {epoch_acc/BATCHES:.3f}')

DEBUT DU TRAVAIL SUR LA DATA DU TEST

In [ ]:
#initialisation du data de test
test_data = pd.read_csv('/kaggle/input/quora-insincere-questions-classification/test.csv')
sentences, targets = test_data['question_text'], []
TEST_BATCHES = (len(sentences)+BATCH_SIZE-1)//BATCH_SIZE

In [ ]:
test_data.head()

In [ ]:
min, avrg, max = configure_sentence_statistic(sentences)

print('minimum sentence length {} - average sentence length {} - maximum sentence length {}'.format(min, avrg, max))

In [ ]:
len(sentences), len(targets), TEST_BATCHES

PREDIT LA DATA DU TEST SUIVANT NOTRE MODELE

In [ ]:
model.eval()
with torch.no_grad():
    for b in range(TEST_BATCHES):
        #obtenir le bash courant
        X_batch = torch.FloatTensor(configure_word2vecs(sentences[BATCH_SIZE*b:BATCH_SIZE*(b+1)], word2vecs)).to(gpu)
        
        # predit bach selon notre modele entraine
        trg = torch.round(torch.sigmoid(model(X_batch))).cpu().numpy().squeeze()
        targets.extend(trg)
        
        if b == 0 or (b+1) % 100 == 0: print(f'Batch {(b+1)+0:04} predicted')

In [ ]:
# enregistre les donnees a transmetre 
test_targets = (np.array(targets) >= 0.5).astype(np.int)

submit = pd.DataFrame({"qid": test_data['qid'], "prediction": test_targets})
submit.to_csv("submission.csv", index=False)

In [ ]:
# Affichage des resultats
submit.head()

comme il parrait de notre prediction il est comme les donnees d entrainement ,c est bon

****voyons quelque exemple de nos prediction

In [ ]:
configure_target_statistic(submit['prediction'])

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=0)

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=1)

il parrait que notre modele marche bien

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=1, min_len=100, max_len=150)

In [ ]:
filter_and_display_data(sentences, np.asarray(targets, dtype='int'), target=1, min_len=100, max_len=150)

..s il n ya plus de longue sequence c est ok